In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Dense,Flatten,Input,Dropout
from keras.models import Sequential
from keras.models import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
#加载数据集
from keras.datasets import mnist
import numpy as np
import cv2
import h5py


inputshape = (224,224)
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, y_train = X_train[:1000], y_train[:1000]#训练集1000条
X_test, y_test = X_test[:100], y_test[:100]#测试集100条

#变换图像size
X_train = [cv2.cvtColor(cv2.resize(i, inputshape), cv2.COLOR_GRAY2RGB)
           for i in X_train]
X_test = [cv2.cvtColor(cv2.resize(i, inputshape), cv2.COLOR_GRAY2RGB)
           for i in X_test]
X_train = np.concatenate([arr[np.newaxis] for arr in X_train]).astype('float32')
X_test = np.concatenate([arr[np.newaxis] for arr in X_test]).astype('float32')

#数据归一化处理
X_train = preprocess_input(np.array(X_train))
X_test = preprocess_input(np.array(X_test))
print(y_train.shape)






 5734400/11490434 [=============>................] - ETA: 2:03

In [3]:
#搭建神经网络
vgg_model = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))
add_model = Sequential()
add_model.add(Flatten(input_shape=vgg_model.output_shape[1:]))
#add_model.add(Dense(4096,activation='relu'))
add_model.add(Dense(1096,activation='relu'))
add_model.add(Dropout(0.5))
add_model.add(Dense(10,activation='softmax'))
model = Model(vgg_model.input,add_model(vgg_model.output))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
            metrics=['accuracy'])
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
__________________________________________________________________________

In [9]:
from keras.utils import to_categorical
train_datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)

test_datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)

#对label进行onehot编码
y_train_ohe = to_categorical(y_train,num_classes=10)
y_test_ohe = to_categorical(y_test,num_classes=10)

batch_size = 16 # tune it
epochs = 10 # increase it
#train_datagen = train_datagen.flow(X_train, y_train_ohe, batch_size=batch_size)
#test_datagen = test_datagen.flow(X_test,y_test_ohe,batch_size=batch_size)
model.fit(X_train,y_train_ohe,batch_size=16,nb_epoch=10,validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 2506s - loss: 2.1050 - acc: 0.4000 - val_loss: 0.6301 - val_acc: 0.8400
Epoch 2/10
800/800 [==============================] - 2621s - loss: 0.5501 - acc: 0.8337 - val_loss: 0.2880 - val_acc: 0.9200
Epoch 3/10
800/800 [==============================] - 2685s - loss: 0.2428 - acc: 0.9325 - val_loss: 0.2508 - val_acc: 0.9450
Epoch 4/10
800/800 [==============================] - 2590s - loss: 0.1294 - acc: 0.9663 - val_loss: 0.1915 - val_acc: 0.9550
Epoch 5/10
800/800 [==============================] - 2586s - loss: 0.0786 - acc: 0.9712 - val_loss: 0.1851 - val_acc: 0.9550
Epoch 6/10
800/800 [==============================] - 2412s - loss: 0.0903 - acc: 0.9700 - val_loss: 0.2334 - val_acc: 0.9550
Epoch 7/10
800/800 [==============================] - 2409s - loss: 0.0734 - acc: 0.9688 - val_loss: 0.1931 - val_acc: 0.9500
Epoch 8/10
800/800 [==============================] - 2406s - loss: 0.03

In [10]:
model.save('mnist.h5')

In [14]:
import pandas as pd 
from keras.models import load_model
import cv2
from PIL import Image
import numpy as np
from keras.applications.imagenet_utils import preprocess_input

df = pd.read_csv('test.csv')
a = np.array(df.loc[0:])
a = np.expand_dims(a,axis=1)
X_test = a.reshape((28000,28,28)).astype('float32')

inputshape = (224,224)
X_test = X_test[:1000]
X_test = [cv2.cvtColor(cv2.resize(i, inputshape), cv2.COLOR_GRAY2RGB)
           for i in X_test]
print(123)
X_test = np.array(X_test).astype('float32')
X_test = preprocess_input(X_test)
print(X_test.shape)



123
(1000, 224, 224, 3)


In [15]:
model = load_model('mnist.h5')
y_pre = model.predict(X_test)
print(type(y_pre),y_pre.shape)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
<class 'numpy.ndarray'> (1000, 10)
